<a href="https://colab.research.google.com/github/gpt2go/llm2finetune/blob/main/Vicuna_7B_Instruct_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Retrained by gpt2go (Aaron Hung) 2023.Aug.3

# Install

In [28]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [29]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [30]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [31]:
!pip install transformers[sentencepiece]

# Import

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [34]:
model_id = "CleverShovel/vicuna-7b-v1.3-sharded-bf16" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: ignored

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3508801536 || trainable%: 0.23907331075678143


# Prepare Data

In [12]:
from datasets import load_dataset

data = load_dataset("zadhart/ESG_Chat")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].map(lambda x: {"input_text": x['Perguntas']  + "\n" + x['Respostas']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

In [14]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [15]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [16]:
def generate(index):

  example_text = data['train'][index]['Perguntas']
  correct_answer = data['train'][index]['Respostas']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  predict = tokenizer.decode(output[0], skip_special_tokens=True)
  predict = predict[len(example_text)+1:]
  print("Answer:")
  print(predict)

  print("Best Answer:")
  print(correct_answer)

  print()

In [17]:
generate(10)

Question:
Define Corporate Social Responsibility in ESG context


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Answer:
Corporate Social Responsibility (CSR) refers to a company' The 2018-2023 World Outlook for Automotive Batteries
This study covers the world outlook for automotive batteries across more than 190 countries. For each year reported, estimates are given for the latent demand, or potential industry earnings (P.I.E.), for the country in question (in millions of U.S. dollars), the percent share the country is of the region, and of the globe.
This report provides you with valuable data and insights about the potential, opportunities, and challenges of the automotive batteries market across
Best Answer:
Corporate Social Responsibility (CSR) in the ESG (Environmental Social and Governance) context is a framework that promotes responsible business practices that promote sustainable economic growth while taking into account broader social and environmental impacts. CSR in the ESG context requires companies to consider their impact on the environment social issues and governance practices.



In [18]:
import random
random.seed(8)
rand_q = random.sample(list(range(len(data['train']))),10)

In [19]:
def get_response_set(model):
   predictions = []

   for index in rand_q:
      example_text = data['train'][index]['Perguntas']
      encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
      output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
      predict = tokenizer.decode(output[0], skip_special_tokens=True)
      predict = predict[len(example_text)+1:]
      predictions.append(predict)

   return predictions


In [20]:
predictions = get_response_set(model)

KeyboardInterrupt: ignored

In [21]:
predictions

NameError: ignored

In [ ]:
reference = data["train"]["Respostas"]

In [ ]:
references = []
questions = []
for i in rand_q:
   references.append(reference[i])
   questions.append(data['train'][i]['Perguntas'])

references

['Green pricing programs are programs offered by electricity providers that allow customers to pay a premium for electricity that is generated from renewable energy sources such as wind or solar power. These programs are often offered as a way to address environmental social and governance (ESG) concerns by reducing carbon emissions and promoting sustainable energy production. The premiums paid by customers are often used to purchase renewable energy certificates (RECs) or fund additional renewable energy projects. By participating in green pricing programs customers can demonstrate their commitment to sustainability and support the development of renewable energy resources which in turn can help to mitigate climate change and promote a more sustainable future.',
 'Green financing is a type of financing that is focused on environmentally sustainable projects and initiatives. It is guided by environmental social and governance (ESG) considerations and aims to support projects which have

In [ ]:
import pandas as pd

def compare_df(question,pred,ref) -> pd.DataFrame:
   info = {
      "quetsion" : question,
      "response" : pred,
      "answer" : ref,
   }

   return pd.DataFrame(info)


df = compare_df(questions,predictions,references)

In [ ]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4dfcb0aca65e72e9359b61ee24420661cec7cd8c12f4393508512d2208f5f730
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
df.head()

,quetsion,response,answer
0,Define green pricing programs in ESG context,Identify key ESG issues relevant to green pric...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the financing of pro...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security refers to the protection of sens...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,The living wage is the minimum amount of incom...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,"Biofuels are renewable, low-carbon energy sour...",Biofuels are renewable energy sources derived ...


In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions,references=references,use_stemmer=True)
results

{'rouge1': 0.4521135134161909,
 'rouge2': 0.16549681545190473,
 'rougeL': 0.27429576028322344,
 'rougeLsum': 0.3053308783332366}

# Training

In [23]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=30,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<ipython-input-14-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
1,1.477900
2,1.446700
3,1.443000
4,1.400400
5,1.310200
6,1.277300
7,1.220000
8,1.198000
9,1.145900
10,1.148100


TrainOutput(global_step=30, training_loss=1.0852774620056151, metrics={'train_runtime': 1331.9626, 'train_samples_per_second': 1.441, 'train_steps_per_second': 0.023, 'total_flos': 9494392398151680.0, 'train_loss': 1.0852774620056151, 'epoch': 10.0})

# Example After Fine Tuning

In [24]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              

In [25]:
generate(39)

Question:
Define emissions in ESG context
Answer:
Emissions in the ESG (Environmental Social and Governance) context refer to the release of pollutants or greenhouse gases into the atmosphere that contribute to climate change and environmental degradation. These emissions can come from a variety of sources including industrial facilities power plants transportation and agriculture. In the ESG context emissions are considered a critical issue as they have a direct impact on the environment and human health and can also affect a company's reputation and financial performance. Companies are increasingly being held accountable for their emissions and are expected to take steps to reduce their environmental impact. This can include implementing sustainable practices such as reducing energy consumption and using renewable energy sources. Companies
Best Answer:
Emissions in the ESG (Environmental Social and Governance) context refer to the release of harmful gases or pollutants into the envir

In [26]:
prediction2 = get_response_set(model)

In [ ]:
prediction2

['Green pricing programs are a type of environmental social and governance (ESG) initiative that allows customers to pay a premium for electricity that is generated from renewable sources such as wind solar or hydroelectric power. The premium is typically passed on to the utility company which then invests in renewable energy projects to reduce the carbon footprint of the power grid.\n\nGreen pricing programs are designed to encourage customers to adopt sustainable energy practices and reduce their environmental impact. By choosing to participate in a green pricing program customers can support the transition to a low-carbon energy system reduce their carbon footprint and contribute to a more sustainable future.\n\nIn addition to supporting renew',
 'Green financing refers to the investment of financial resources in projects or companies that promote environmental social and governance (ESG) principles. In the context of ESG green financing refers to the investment of capital in compan

In [27]:
results2 = rouge.compute(predictions=prediction2,references=references,use_stemmer=True)

NameError: ignored

In [ ]:
results2

{'rouge1': 0.5697584496189545,
 'rouge2': 0.2639641892305975,
 'rougeL': 0.3603096245769519,
 'rougeLsum': 0.39111540660527333}

In [ ]:
bleu = evaluate.load('bleu')
results_bleu = bleu.compute(predictions=predictions, references=references)
results_bleu2 = bleu.compute(predictions=prediction2, references=references)

print(results_bleu)
print(results_bleu2)

{'bleu': 0.07690578149807137, 'precisions': [0.4226713532513181, 0.13386524822695037, 0.0518783542039356, 0.01895306859205776], 'brevity_penalty': 0.8904814832507468, 'length_ratio': 0.8960629921259843, 'translation_length': 1138, 'reference_length': 1270}
{'bleu': 0.20775165644711646, 'precisions': [0.565587734241908, 0.26288659793814434, 0.16031195840554593, 0.10839160839160839], 'brevity_penalty': 0.9214822901003235, 'length_ratio': 0.9244094488188976, 'translation_length': 1174, 'reference_length': 1270}


In [ ]:
toxicity = evaluate.load("toxicity")
results_tox = toxicity.compute(predictions=predictions, aggregation="ratio")
results_tox2 = toxicity.compute(predictions=prediction2, aggregation="ratio")
print(results_tox)
print(results_tox2)

{'toxicity_ratio': 0.0}
{'toxicity_ratio': 0.0}


In [ ]:
df.insert(loc = 2, column="vicuna7b_model_response", value=prediction2)

In [ ]:
df.insert(loc = 3, column="vicuna7b_response", value=predictions3)

In [ ]:
df

,quetsion,response,vicuna7b_model_response,answer
0,Define green pricing programs in ESG context,Identify key ESG issues relevant to green pric...,Green pricing programs are a type of environme...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the financing of pro...,Green financing refers to the investment of fi...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security refers to the protection of sens...,In the context of Environmental Social and Gov...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,The living wage is the minimum amount of incom...,In the context of ESG (Environmental Social an...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,"Biofuels are renewable, low-carbon energy sour...",Biofuels are renewable energy sources that are...,Biofuels are renewable energy sources derived ...
5,Define Refinitiv Scoring in ESG context,\n* Refinitiv Scoring is a proprietary ESG sco...,Refinitiv Scoring is a methodology used to eva...,Refinitiv Scoring in ESG context refers to an ...
6,Define corporate governance in ESG context,Corporate governance refers to the system of r...,Corporate governance in the ESG (Environmental...,Corporate governance in ESG (Environmental Soc...
7,Define governance through exit in ESG context,"In the context of Environmental, Social and Go...",Governance through exit in the ESG (Environmen...,Governance through exit in ESG context refers ...
8,Define stakeholders in ESG context,Explain the role of stakeholders in ESG\nDiscu...,Stakeholders in the ESG (Environmental Social ...,Stakeholders in an ESG (Environmental Social a...
9,Define floods in ESG context,Floods are a natural hazard that can cause sig...,Floods in the ESG (Environmental Social and Go...,In the context of ESG (Environmental Social an...


In [ ]:
c1 = "rouge1"
c2 = "rouge2"
cL = "rougeL"
cLsum = "rougeLsum"
df_score = pd.DataFrame({
    c1 : [results[c1], results2[c1], results3[c1]],
    c2 : [results[c2], results2[c2], results3[c2]],
    cL : [results[cL], results2[cL], results3[cL]],
    cLsum : [results[cLsum], results2[cLsum], results3[cLsum]],
})
df_score.index = ["falcon7b" , "ESG_model", "vicuna7b"]
df_score

,rouge1,rouge2,rougeL,rougeLsum
falcon7b,0.466976,0.145024,0.273879,0.294289
ESG_model,0.525273,0.203351,0.325743,0.354960
vicuna7b,0.457632,0.168178,0.277941,0.301291


# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.save_pretrained("vicuna-7b-instruct-ft-adapters-ESG-chatting", push_to_hub=True) # save LORA adapters
model.push_to_hub("vicuna-7b-instruct-ft-adapters-ESG-chatting")

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/vicuna-7b-instruct-ft-adapters-ESG-chatting/commit/39181a93826799ba097346dcd71d569fa0844a16', commit_message='Upload model', commit_description='', oid='39181a93826799ba097346dcd71d569fa0844a16', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig, LoraConfig
from transformers import AutoModelForCausalLM

config = LoraConfig.from_pretrained("FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting")
# reload the base model (you might need pro subscription for this)

In [ ]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    "FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting",
)

In [ ]:
model = model.merge_and_unload() # merge adapters with

ValueError: ignored

In [ ]:
model.push_to_hub("falcon-7b-instruct-ft")

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/93f6d84447abd80fff5ff062c81dc86b0ac72e82', commit_message='Upload RWForCausalLM', commit_description='', oid='93f6d84447abd80fff5ff062c81dc86b0ac72e82', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.push_to_hub("falcon-7b-instruct-ft", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/ae7eca5a54426134be2babe2239380487dad3306', commit_message='Upload tokenizer', commit_description='', oid='ae7eca5a54426134be2babe2239380487dad3306', pr_url=None, pr_revision=None, pr_num=None)